# 1 - Packaging

## Absolute imports
Imports which use a full path to the module
* from reader.reader import Reader

In [1]:
import urllib  # package == directory
import urllib.request  # module == single file
print(type(urllib), type(urllib.request))
urllib.__path__  # Where urllib searches for nested modules

<class 'module'> <class 'module'>


['C:\\Anaconda3\\lib\\urllib']

In [3]:
# sys.path
# Python checks sys.path - list of directories Python searches
# for modules
import sys
sys.path

['',
 'C:\\Anaconda3\\python35.zip',
 'C:\\Anaconda3\\DLLs',
 'C:\\Anaconda3\\lib',
 'C:\\Anaconda3',
 'c:\\anaconda3\\lib\\site-packages\\setuptools-20.3-py3.5.egg',
 'C:\\Anaconda3\\lib\\site-packages',
 'C:\\Anaconda3\\lib\\site-packages\\Sphinx-1.3.5-py3.5.egg',
 'C:\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Spiridonov\\.ipython']

In [7]:
sys.path[0]  # '' - look in current directory first
# sys.path.append('not_searched')  # add entries to sys.path

''

In [12]:
# PYTHONPATH - environment variable listing paths to look for modules
!echo %PYTHONPATH%

%PYTHONPATH%


In [1]:
# path_entry/my_package/__init__.py
# path_entry -- must be in sys.path
# __init__.py makes package a module
import reader
print(type(reader),reader.__file__)
import reader.reader
reader.reader.__file__

<class 'module'> c:\code\sandbox\anaconda\[pluralsight] Python Beyond the Basics\reader\__init__.py


'c:\\code\\sandbox\\anaconda\\[pluralsight] Python Beyond the Basics\\reader\\reader.py'

In [3]:
import reader
# Added "from reader.reader import Reader" to __init__.py
# The code from __init__.py gets executed..
# ..and the result is bound to reader namespace!
# otherwise would have to type r = reader.reader.Reader('reader/reader.py')  
r = reader.Reader('reader/__init__.py')
print(r.read())
r.close()

# ..or even more concise - import into global namespace
# (if __init__.py was empty, one would have to specify the file in the package, i.e. "from reader.reader import Reader")
from reader import Reader
r = Reader('reader/__init__.py')
print(r.read())
r.close()

# print('reader is being imported!')
from reader.reader import Reader  # hoist to top-level package
# print('reader is being imported!')
from reader.reader import Reader  # hoist to top-level package


In [5]:
import reader
import reader.compressed
import reader.compressed.bzipped

In [17]:
# Create compressed files
!python -m reader.compressed.bzipped test.bz2 data compressed with bz2
!python -m reader.compressed.gzipped test.gz data compressed with gzip

import reader
r = reader.Reader('test.bz2')
print(r.read())
r.close()

r = reader.Reader('test.gz')
print(r.read())
r.close()

data compressed with bz2
data compressed with gzip


## Relative imports
Imports which use a relative path to modules **in the same package**
* from .reader import Reader
* . - same directory, .. - parent directory, etc.

farm / bovine / {cow.py, common.py}
* In cow.py one could use one of the following
* Absolute: `from farm.bovine.common import ruminate`
* Relative: 
    * `from .common import ruminate`
    * `from . import common` => use commmon.function() 

## \__all\__ attribute
List of attribute names imported via `from module import *`

In [10]:
from reader.compressed import *
locals()
print(type(bz2_opener),type(gzip_opener))

<class 'function'> <class 'function'>


## namespace packages
Packages split accross several directories (PEP420) 
* have no `__init__.py`
* This avoids complex initialization oredering problems.
* How does Python finds namespace packages then? **When asked to import name "foo", Python does the following:**
    * scans all entries in sys.path
    * if `foo/__init__.py` -> normal package is loaded
    * if foo.py, then this module is loaded
    * otherwise, all matching dirs in sys.path are considered
    part of the namespace package

In [13]:
# split_farm is a namespace package
# path1/split_farm/bovine/{__init__.py, cow.py, string.py}
# path2/split_farm/bird/{__init__.py, chicken.py, turkey.py}
import sys
# ensure both path1 and path2 are in sys.path
sys.path.extend(['path1', 'path2'])

# Python loads all matching dirs according to the algorithm
import split_farm

split_farm.__path__
import split_farm.bird  # path2/split_farm/bird
import split_farm.bovine  # path1/split_farm/bovine

ImportError: No module named 'split_farm'

## Executable directories
Directories containing entry point for Python execution. One step further - executable zip file.

In [24]:
!python reader_exec test.gz  
    # ==> can put __main__.py in a directory

data compressed with gzip


In [23]:
!python reader/compressed

executing __main__.py with name __main__


## Recommended project structure
* project_name
    * licence
    * `__main__.py`
    * project_name
        * `__init__.py`
        * more_source.py
        * subpackage1
            * `__init__.py`
        * test
            * `__init__.py`
            * test_code.py
    * setup.py

## Singleton
Implement Singleton as a module-level attribute.
* Modules are only executed once, when first imported.
* Module initialization order is well defined

**registry.py**

```
_registry = []

def register(name):
    _register.append(name)
    
def registered_names():
    return iter(_registry)
```

**use_registry.py**

```
import registry

registry.register('my name')
for name in registry.registered_names():
    print(name)
```
